<a name="top"> <h1>02. Neuronal Networks</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>


<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[Introducción](#introduccion) 

[1. Librerias utilizadas y funciones](#librerias) 

[2. Lectura del dataframe y preparación de los datos](#lectura) 

   - 2.1. Lectura del DF
   - 2.2. Preparación de los datos

[3. Modelo de la red neuronal](#modelo) 

[4. Backup del modelo](#backup) 

<hr style="border:1px solid gray">

# <a name="introduccion"> Introducción <a>


En este tercer notebook, tras haber preprocesado los datos y evaluado diferentes modelos de Machine Learning, exploraremos las redes neuronales con el objetivo de superar los resultados obtenidos en `1_MLModels`

Para ello nos ayudaremos principalmente de la libreria TensorFlow, la cual nos permite de una forma secilla, crear arquitecutras de redes neuronales y modificar los hiperparámetros para ajustar el modelo, además de introducir técnicas de regularización.

Una vez obtengamos los resultados y seleccionemos aquel que mejor resultado haya arrojado, procederemos a seleccionarlo ya sea de ML o DL, para ponerlo en producción.

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para la creación de redes neuronales:

In [3]:
# Import basic libraries.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
import string
import pickle

# Import neural network libraries.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# <a name="lectura"> 2. Lectura del dataframe y preparación de los datos<a>


## 2.1. Lectura del DF

In [4]:
#Import parquet file.

reviews = pd.read_parquet('../../data/processed/reviews.parquet')

# Show the head of the dataframe.

reviews.head()

,reviewText,overall
0,amazon shopping amazoncom gift cards christmas...,1
1,got gift card friend best site much choose gre...,5
2,arent going save trees people complaining pape...,5
3,always get someone something amazon safety net...,5
4,take 50 dollars good money limitations turn am...,1


## 2.2. Preparación de los datos

Al igual que en el notebook anterior, preparamos los datos para que puedan ser introducidos dentro de los modelos, separando en X el texto y en y las puntuaciones. Y modificando el formato para que puedan ser introducidos dentro de la red.

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



# Dividir los datos en características y etiquetas
X = reviews['reviewText'].values
y = reviews['overall'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Crear el Tokenizer y ajustarlo solo a las palabras del conjunto de entrenamiento
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convertir las secuencias de palabras en secuencias numéricas
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Aplicar padding para que todas las secuencias tengan la misma longitud
max_sequence_length = max([len(sequence) for sequence in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Convertir las etiquetas a one-hot encoding
num_classes = 6
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes)

# Crear el modelo de redes neuronales
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
#model.fit(X_train_padded, y_train_encoded, validation_data=(X_test_padded, y_test_encoded), epochs=10, batch_size=32)


In [6]:
# Load the model.

model = tf.keras.models.load_model('../../models/nn_reviews.h5', compile=False)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load the tokenizer that trained the Neuronal Network.

with open('../../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Predict the class of the review.

input_text = pd.Series("The best product, Incredible")[0]

# Convert the text to sequence of words

X_test_sequences = tokenizer.texts_to_sequences([input_text])

# Pad the sequence

X_test_padded = pad_sequences(X_test_sequences, maxlen=len(X_test_sequences), padding='post')

# Make predictions

predictions = model.predict(X_test_padded)

# Get the sentiment with the highest probability

sentiment = np.argmax(predictions)

sentiment

1/1 [==============================] - 2s 2s/step


5

In [7]:
# Predict x_test

X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_test_padded = pad_sequences(X_test_sequences, maxlen=len(X_test_sequences), padding='post')

predictions = model.predict(X_test_padded)

sentiments = np.argmax(predictions, axis=1)

# F2 score

from sklearn.metrics import fbeta_score

fbeta_score(y_test, sentiments, average='micro', beta=2)



  1/920 [..............................] - ETA: 7:07:15

KeyboardInterrupt: 

In [3]:
# Define the X and y variables.

X = reviews['reviewText'].values
y = reviews['overall'].values

# Modify the y variable to be a categorical variable.

y = tf.keras.utils.to_categorical(y-1, num_classes=5)

# Split the data into train and test sets.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Define the tokenizer.

tokenizer = Tokenizer(num_words=10000)

# Fit the tokenizer.

tokenizer.fit_on_texts(X_train)

# Transform the train and test sets.

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Add padding to the train and test sets.

vocab_size = len(tokenizer.word_index) + 1

maxlen = 300

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)



In [4]:
# Save the tokenizer in models.

with open('../../models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


# <a name="modelo"> 3. Modelo de la red neuronal<a>

Tras hacer diferentes pruebas fuera del notebook, ponemos a prueba el que mejor resultado nos ha dado:

In [5]:
# Define the model.

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=300),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Compile the model.
z
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# Fit the model.

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

Epoch 1/10
920/920 [==============================] - 550s 594ms/step - loss: 0.5846 - accuracy: 0.8596 - val_loss: 0.5708 - val_accuracy: 0.8617
Epoch 2/10
920/920 [==============================] - 567s 616ms/step - loss: 0.5747 - accuracy: 0.8604 - val_loss: 0.5695 - val_accuracy: 0.8617
Epoch 3/10
920/920 [==============================] - 598s 650ms/step - loss: 0.5746 - accuracy: 0.8605 - val_loss: 0.5693 - val_accuracy: 0.8617
Epoch 4/10
920/920 [==============================] - 720s 782ms/step - loss: 0.5745 - accuracy: 0.8604 - val_loss: 0.5694 - val_accuracy: 0.8617
Epoch 5/10
920/920 [==============================] - 562s 611ms/step - loss: 0.5745 - accuracy: 0.8605 - val_loss: 0.5708 - val_accuracy: 0.8617
Epoch 6/10
920/920 [==============================] - 568s 617ms/step - loss: 0.5743 - accuracy: 0.8605 - val_loss: 0.5701 - val_accuracy: 0.8617
Epoch 7/10
920/920 [==============================] - 560s 609ms/step - loss: 0.5764 - accuracy: 0.8600 - val_loss: 0.5697 -

# <a name="backup"> 4. Backup del modelo<a>

Dado que es el modelo que mejor resultado nos ha dado, procedemos a guardarlo para posteriormente ponerlo en producción.

In [7]:
# Guardar el modelo en un archivo
model.save("../../models/nn_reviews.h5")

In [1]:
# Load the model.

model = tf.keras.models.load_model('../../models/nn_reviews.h5', compile=False)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load the tokenizer that trained the Neuronal Network.

with open('../../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Predict the class of the review.

input_text = pd.Series("Horrible")[0]

# Convert the text to sequence of words

X_test_sequences = tokenizer.texts_to_sequences([input_text])

# Pad the sequence

X_test_padded = pad_sequences(X_test_sequences, maxlen=len(X_test_sequences), padding='post')

# Make predictions

predictions = model.predict(X_test_padded)

# Get the sentiment with the highest probability

sentiment = np.argmax(predictions)

sentiment

NameError: name 'tf' is not defined